In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pprint
from scipy import stats

from feature_engine import transformation as vt
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from pprint import pprint

In [2]:
pd.set_option("display.max_columns",None)

In [3]:
train_set=pd.read_csv(r'C:\Users\Prawin\Documents\ML PROJECT\existing_base_train.csv')

In [4]:
train_set.head(2)

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,year_last_moved,TVarea,post_code,post_area,Average_Credit_Card_Transaction,Balance_Transfer,Term_Deposit,Life_Insurance,Medical_Insurance,Average_A/C_Balance,Personal_Loan,Investment_in_Mutual_Fund,Investment_Tax_Saving_Bond,Home_Loan,Online_Purchase_Amount,gender,region,Investment_in_Commudity,Investment_in_Equity,Investment_in_Derivative,Portfolio_Balance,Revenue_Grid
0,5466,2,31-35,Partner,Professional,Professional,Own Home,">=35,000",No,No,1981,Meridian,M51 0GU,M51,26.98,29.99,312.25,299.79,88.72,108.85,175.43,134.35,8.98,55.44,7.68,Female,North West,151.55,81.79,136.02,360.37,2
1,9091,Zero,45-50,Partner,Secretarial/Admin,Professional,Own Home,">=35,000",No,No,1997,Meridian,L40 2AG,L40,35.98,74.48,0.00,99.96,10.99,48.45,15.99,0.00,0.00,0.00,18.99,Female,North West,44.28,13.91,29.23,89.22,2


In [5]:
y_main=train_set['Revenue_Grid']
x_main=train_set.drop(columns=['Revenue_Grid','post_code','REF_NO'])

# Removing Unknown

In [6]:
columns=x_main.columns
columns

Index(['children', 'age_band', 'status', 'occupation', 'occupation_partner',
       'home_status', 'family_income', 'self_employed',
       'self_employed_partner', 'year_last_moved', 'TVarea', 'post_area',
       'Average_Credit_Card_Transaction', 'Balance_Transfer', 'Term_Deposit',
       'Life_Insurance', 'Medical_Insurance', 'Average_A/C_Balance',
       'Personal_Loan', 'Investment_in_Mutual_Fund',
       'Investment_Tax_Saving_Bond', 'Home_Loan', 'Online_Purchase_Amount',
       'gender', 'region', 'Investment_in_Commudity', 'Investment_in_Equity',
       'Investment_in_Derivative', 'Portfolio_Balance'],
      dtype='object')

In [7]:
for z in columns:
    if x_main[x_main[z]=='Unknown'][z].count() !=0:
        print('Column is:',z)
        print(x_main[x_main[z]=='Unknown'][z].count())

Column is: age_band
44
Column is: status
40
Column is: occupation
449
Column is: occupation_partner
1942
Column is: family_income
108
Column is: TVarea
154
Column is: gender
31
Column is: region
866


In [8]:
columns_2=['age_band','status','family_income','TVarea','gender','region']
for y in columns_2:
    z=str(x_main[y].mode())
    x_main[y]=x_main[y].str.replace('Unknown',z)


In [9]:
for z in columns:
    if x_main[x_main[z]=='Unknown'][z].count() !=0:
        print('Column is:',z)
        print(x_main[x_main[z]=='Unknown'][z].count())

Column is: occupation
449
Column is: occupation_partner
1942


In [10]:
retired_age_band=['61-65','65-70','71+']

In [11]:
x_main.loc[(x_main['occupation']=='Unknown') & (x_main['age_band'].isin(retired_age_band)), 'occupation']='Retired'
x_main.loc[(x_main['occupation']=='Unknown'), 'occupation']='Professional'
x_main.loc[(x_main['occupation_partner']=='Unknown') & (x_main['age_band'].isin(retired_age_band)), 'occupation_partner']='Retired'
x_main.loc[(x_main['occupation_partner']=='Unknown'), 'occupation_partner']='Professional'

In [12]:
x_main['age_band'].value_counts()

45-50                        1098
41-45                         903
36-40                         895
55-60                         865
31-35                         840
51-55                         833
26-30                         735
61-65                         700
65-70                         468
22-25                         356
71+                           337
18-21                          50
0    45-50\ndtype: object      44
Name: age_band, dtype: int64

In [13]:
x_main['age_band'] = x_main['age_band'].str.replace('0    45-50\ndtype: object','45-50')

# Converting columns to comparable forms

In [14]:
x_main['age_band'] = x_main['age_band'].str.replace('45-50','50',regex=False)
x_main['age_band'] = x_main['age_band'].str.replace('41-45','45',regex=False)
x_main['age_band'] = x_main['age_band'].str.replace('36-40','40',regex=False)
x_main['age_band'] = x_main['age_band'].str.replace('55-60','60',regex=False)
x_main['age_band'] = x_main['age_band'].str.replace('31-35','35',regex=False)
x_main['age_band'] = x_main['age_band'].str.replace('51-55','55',regex=False)
x_main['age_band'] = x_main['age_band'].str.replace('26-30','30',regex=False)
x_main['age_band'] = x_main['age_band'].str.replace('61-65','65',regex=False)
x_main['age_band'] = x_main['age_band'].str.replace('65-70','70',regex=False)
x_main['age_band'] = x_main['age_band'].str.replace('22-25','25',regex=False)
x_main['age_band'] = x_main['age_band'].str.replace('71+','71',regex=False)
x_main['age_band'] = x_main['age_band'].str.replace('18-21','21',regex=False)



In [15]:
x_main[['age_band']] = x_main[['age_band']].apply(pd.to_numeric)

In [16]:
x_main['family_income'].unique()

array(['>=35,000', '<22,500, >=20,000', '<25,000, >=22,500',
       '<30,000, >=27,500', '<20,000, >=17,500', '<27,500, >=25,000',
       '< 4,000', '<15,000, >=12,500', '<17,500, >=15,000',
       '< 8,000, >= 4,000', '<12,500, >=10,000', '<10,000, >= 8,000',
       '0    >=35,000\ndtype: object'], dtype=object)

In [17]:
salary_1=['<22,500, >=20,000', '<25,000, >=22,500','<30,000, >=27,500', '<20,000, >=17,500', '<27,500, >=25,000','< 4,000', '<15,000, >=12,500', '<17,500, >=15,000',
       '< 8,000, >= 4,000', '<12,500, >=10,000', '<10,000, >= 8,000']

In [18]:
x_main['family_income'] = x_main['family_income'].str.replace('>=35,000','35000',regex=False)
x_main['family_income'] = x_main['family_income'].str.replace('<22,500, >=20,000','22500',regex=False)
x_main['family_income'] = x_main['family_income'].str.replace('<25,000, >=22,500','25000',regex=False)
x_main['family_income'] = x_main['family_income'].str.replace('<30,000, >=27,500','30000',regex=False)
x_main['family_income'] = x_main['family_income'].str.replace('<20,000, >=17,500','20000',regex=False)
x_main['family_income'] = x_main['family_income'].str.replace('<27,500, >=25,000','27500',regex=False)
x_main['family_income'] = x_main['family_income'].str.replace('< 4,000','4000',regex=False)
x_main['family_income'] = x_main['family_income'].str.replace('<15,000, >=12,500','15000',regex=False)
x_main['family_income'] = x_main['family_income'].str.replace('<17,500, >=15,000','17500',regex=False)
x_main['family_income'] = x_main['family_income'].str.replace('< 8,000, >= 4,000','8000',regex=False)
x_main['family_income'] = x_main['family_income'].str.replace('<12,500, >=10,000','12500',regex=False)
x_main['family_income'] = x_main['family_income'].str.replace('<10,000, >= 8,000','10000',regex=False)
x_main['family_income'] = x_main['family_income'].str.replace('0    35000\ndtype: object','35000',regex=False)



In [19]:
x_main[['family_income']] = x_main[['family_income']].apply(pd.to_numeric)


In [20]:
x_main['status'] = x_main['status'].str.replace('0    Partner\ndtype: object','Partner',regex=False)
x_main.loc[x_main['year_last_moved']==0,'year_last_moved']=1997
x_main['TVarea'] = x_main['TVarea'].str.replace('0    Central\ndtype: object','Central',regex=False)
x_main['gender'] = x_main['gender'].str.replace('0    Female\ndtype: object','Female',regex=False)
x_main['region'] = x_main['region'].str.replace('0    South East\ndtype: object','South East',regex=False)




In [21]:
x_main.drop(['year_last_moved','children','TVarea'],inplace=True,axis=1)

In [22]:
x_main.drop(['post_area','region'],inplace=True,axis=1)


# Merging columns

In [23]:
# x_main['investments']=x_main['Investment_in_Mutual_Fund']+x_main['Investment_Tax_Saving_Bond']+x_main['Investment_in_Commudity']+x_main['Investment_in_Equity']+x_main['Investment_in_Derivative']+x_main['Term_Deposit']
# x_main['spending']=x_main['Balance_Transfer']+x_main['Online_Purchase_Amount']
# x_main['loans']=x_main['Personal_Loan']+x_main['Home_Loan']
# x_main['insurance']=x_main['Life_Insurance']+x_main['Medical_Insurance']

In [24]:
# x_main.drop(columns=['Life_Insurance','Medical_Insurance','Personal_Loan','Investment_in_Mutual_Fund','Investment_Tax_Saving_Bond','Home_Loan','Investment_in_Commudity','Investment_in_Equity','Investment_in_Derivative','Term_Deposit','Balance_Transfer','Online_Purchase_Amount','post_area','region'],inplace=True)


In [25]:
cat_cols =list(['status', 'occupation', 'occupation_partner', 'home_status',
       'self_employed', 'self_employed_partner', 'gender',
       ])

In [26]:
for c in cat_cols:
    print(x_main[c].value_counts())
    print('='*70)

Partner                 6164
Single/Never Married     881
Divorced/Separated       569
Widowed                  510
Name: status, dtype: int64
Professional         2335
Retired              1862
Secretarial/Admin    1435
Housewife             984
Business Manager      578
Manual Worker         451
Other                 432
Student                47
Name: occupation, dtype: int64
Professional         3103
Retired              2017
Manual Worker        1222
Business Manager      575
Secretarial/Admin     510
Housewife             422
Other                 261
Student                14
Name: occupation_partner, dtype: int64
Own Home                7506
Rent from Council/HA     279
Rent Privately           205
Live in Parental Hom      90
Unclassified              44
Name: home_status, dtype: int64
No     7543
Yes     581
Name: self_employed, dtype: int64
No     7207
Yes     917
Name: self_employed_partner, dtype: int64
Female    6137
Male      1987
Name: gender, dtype: int64


# Label encoding for all the categorical columns

In [27]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for x in cat_cols:
    x_main[x] = le.fit_transform(x_main[x].astype(str))
  

# Skewness of columns

In [28]:
x_main['Portfolio_Balance'].skew(axis=0)

8.895471542761344

In [29]:
x_main['Portfolio_Balance']=np.log(x_main['Portfolio_Balance']+80)

In [30]:
x_main['Portfolio_Balance'].skew(axis=0)

-0.21879018497721892

In [31]:
x_main.columns

Index(['age_band', 'status', 'occupation', 'occupation_partner', 'home_status',
       'family_income', 'self_employed', 'self_employed_partner',
       'Average_Credit_Card_Transaction', 'Balance_Transfer', 'Term_Deposit',
       'Life_Insurance', 'Medical_Insurance', 'Average_A/C_Balance',
       'Personal_Loan', 'Investment_in_Mutual_Fund',
       'Investment_Tax_Saving_Bond', 'Home_Loan', 'Online_Purchase_Amount',
       'gender', 'Investment_in_Commudity', 'Investment_in_Equity',
       'Investment_in_Derivative', 'Portfolio_Balance'],
      dtype='object')

In [32]:
num_cols=['Average_Credit_Card_Transaction', 'Balance_Transfer', 'Term_Deposit',
       'Life_Insurance', 'Medical_Insurance', 'Average_A/C_Balance',
       'Personal_Loan', 'Investment_in_Mutual_Fund',
       'Investment_Tax_Saving_Bond', 'Home_Loan', 'Online_Purchase_Amount',
        'Investment_in_Commudity', 'Investment_in_Equity',
       'Investment_in_Derivative', 'Portfolio_Balance']

skew_values=dict()
for x in num_cols:
    skew_values[x]=x_main[x].skew(axis=0)
        

In [33]:
pprint(skew_values)

{'Average_A/C_Balance': 2.6899852225751015,
 'Average_Credit_Card_Transaction': 3.761152392656973,
 'Balance_Transfer': 8.17373435516367,
 'Home_Loan': 3.9776570354132743,
 'Investment_Tax_Saving_Bond': 3.1917131080167658,
 'Investment_in_Commudity': 4.816122212927679,
 'Investment_in_Derivative': 12.462311620123822,
 'Investment_in_Equity': 11.524906448106663,
 'Investment_in_Mutual_Fund': 9.667247131810205,
 'Life_Insurance': 5.5093788295282105,
 'Medical_Insurance': 3.3619401952736054,
 'Online_Purchase_Amount': 21.763954245754945,
 'Personal_Loan': 26.159595921690393,
 'Portfolio_Balance': -0.21879018497721892,
 'Term_Deposit': 4.174162624313323}


In [34]:
for x in list(skew_values):
    if skew_values[x]>-0.5 and skew_values[x]<0.5:
        del skew_values[x]

In [35]:
pprint(skew_values)

{'Average_A/C_Balance': 2.6899852225751015,
 'Average_Credit_Card_Transaction': 3.761152392656973,
 'Balance_Transfer': 8.17373435516367,
 'Home_Loan': 3.9776570354132743,
 'Investment_Tax_Saving_Bond': 3.1917131080167658,
 'Investment_in_Commudity': 4.816122212927679,
 'Investment_in_Derivative': 12.462311620123822,
 'Investment_in_Equity': 11.524906448106663,
 'Investment_in_Mutual_Fund': 9.667247131810205,
 'Life_Insurance': 5.5093788295282105,
 'Medical_Insurance': 3.3619401952736054,
 'Online_Purchase_Amount': 21.763954245754945,
 'Personal_Loan': 26.159595921690393,
 'Term_Deposit': 4.174162624313323}


In [36]:
## Log transformation 

for z in skew_values:
    x_main[z]=np.log(x_main[z]+1)


In [37]:
skew_values_2=dict()
for x in num_cols:
    skew_values_2[x]=x_main[x].skew(axis=0)
for x in list(skew_values_2):
    if skew_values_2[x]>-0.5 and skew_values_2[x]<0.5:
        del skew_values_2[x]
pprint(skew_values_2)

{'Average_Credit_Card_Transaction': 0.8051792765216884,
 'Home_Loan': 1.2159097449753506,
 'Investment_Tax_Saving_Bond': 1.0793572014516624,
 'Investment_in_Commudity': -0.7501730007028966,
 'Investment_in_Derivative': -0.6849206090542298,
 'Online_Purchase_Amount': 1.4035161557946367,
 'Personal_Loan': 0.9133060427134978,
 'Term_Deposit': 0.542801917935399}


In [38]:
### box cox transformation 

for z in skew_values_2:
    x_main[z],_=stats.boxcox(x_main[z]+1)

In [39]:
skew_values_2=dict()
for x in num_cols:
    skew_values_2[x]=x_main[x].skew(axis=0)
for x in list(skew_values_2):
    if skew_values_2[x]>-0.5 and skew_values_2[x]<0.5:
        del skew_values_2[x]
pprint(skew_values_2)

{'Average_Credit_Card_Transaction': 0.516734650125929,
 'Home_Loan': 0.8433235274720438,
 'Investment_Tax_Saving_Bond': 0.6173105061104122,
 'Online_Purchase_Amount': 0.9133226108581012,
 'Personal_Loan': 0.5892806556126351}


In [40]:
x_main['Average_Credit_Card_Transaction'],_=stats.boxcox(x_main['Average_Credit_Card_Transaction']+1)
x_main['Home_Loan'],_=stats.boxcox(x_main['Average_Credit_Card_Transaction']+1)
x_main['Investment_Tax_Saving_Bond'],_=stats.boxcox(x_main['Average_Credit_Card_Transaction']+1)
x_main['Online_Purchase_Amount'],_=stats.boxcox(x_main['Average_Credit_Card_Transaction']+1)
x_main['Personal_Loan'],_=stats.boxcox(x_main['Average_Credit_Card_Transaction']+1)

In [41]:
print(x_main['Average_Credit_Card_Transaction'].skew(axis=0))
print(x_main['Home_Loan'].skew(axis=0))
print(x_main['Investment_Tax_Saving_Bond'].skew(axis=0))
print(x_main['Online_Purchase_Amount'].skew(axis=0))
print(x_main['Personal_Loan'].skew(axis=0))

0.4958726853368258
0.49148287543865765
0.49148287543865765
0.49148287543865765
0.49148287543865765


# Checking if target variable is balanced

In [42]:
y_main.value_counts()

2    7264
1     860
Name: Revenue_Grid, dtype: int64

In [43]:
7264/860

8.446511627906977

In [44]:
from imblearn.over_sampling import BorderlineSMOTE
ros=BorderlineSMOTE(random_state=0)

In [45]:
x_main_1,y_main_1=ros.fit_resample(x_main,y_main)

In [46]:
y_main_1.value_counts()

2    7264
1    7264
Name: Revenue_Grid, dtype: int64

# Train Test Split-Model

In [47]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x_main_1,y_main_1,random_state=102,test_size=0.25)

In [48]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((10896, 24), (3632, 24), (10896,), (3632,))

In [49]:

from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler()
X_train[num_cols]=mms.fit_transform(X_train[num_cols])
X_test[num_cols]=mms.fit_transform(X_test[num_cols])

In [50]:
from sklearn.linear_model import LogisticRegression

logreg2=LogisticRegression(max_iter=1000)
logreg2.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [52]:
from sklearn.metrics import f1_score

In [53]:
y_pred_train=logreg2.predict(X_train)
y_pred_test=logreg2.predict(X_test)

In [54]:
f1_score(y_train, y_pred_train)

0.6656647807637907

In [55]:
f1_score(y_test, y_pred_test)

0.6777163904235728

# Test Set

In [ ]:
df_test=pd.read_csv(r'C:\Users\Prawin\Documents\ML PROJECT\existing_base_test.csv')


In [ ]:
df_test.head()

In [ ]:
columns_2=['age_band','status','family_income','TVarea','gender']
for y in columns_2:
    z=str(df_test[y].mode())
    df_test[y]=df_test[y].str.replace('Unknown',z)


In [ ]:
columns = df_test.columns

In [ ]:
for z in columns:
    if df_test[df_test[z]=='Unknown'][z].count() !=0:
        print('Column is:',z)
        print(df_test[df_test[z]=='Unknown'][z].count())

In [ ]:
retired_age_band=['61-65','65-70','71+']

In [ ]:
df_test.loc[(df_test['occupation']=='Unknown') & (df_test['age_band'].isin(retired_age_band)), 'occupation']='Retired'
df_test.loc[(df_test['occupation']=='Unknown'), 'occupation']='Professional'
df_test.loc[(df_test['occupation_partner']=='Unknown') & (df_test['age_band'].isin(retired_age_band)), 'occupation_partner']='Retired'
df_test.loc[(df_test['occupation_partner']=='Unknown'), 'occupation_partner']='Professional'

In [ ]:
df_test['age_band'].value_counts()

In [ ]:
df_test['age_band'] = df_test['age_band'].str.replace('0    45-50\ndtype: object','45-50')

# Converting columns to comparable forms

In [ ]:
df_test['age_band'] = df_test['age_band'].str.replace('45-50','50',regex=False)
df_test['age_band'] = df_test['age_band'].str.replace('41-45','45',regex=False)
df_test['age_band'] = df_test['age_band'].str.replace('36-40','40',regex=False)
df_test['age_band'] = df_test['age_band'].str.replace('55-60','60',regex=False)
df_test['age_band'] = df_test['age_band'].str.replace('31-35','35',regex=False)
df_test['age_band'] = df_test['age_band'].str.replace('51-55','55',regex=False)
df_test['age_band'] = df_test['age_band'].str.replace('26-30','30',regex=False)
df_test['age_band'] = df_test['age_band'].str.replace('61-65','65',regex=False)
df_test['age_band'] = df_test['age_band'].str.replace('65-70','70',regex=False)
df_test['age_band'] = df_test['age_band'].str.replace('22-25','25',regex=False)
df_test['age_band'] = df_test['age_band'].str.replace('71+','71',regex=False)
df_test['age_band'] = df_test['age_band'].str.replace('18-21','21',regex=False)



In [ ]:
df_test[['age_band']] =df_test[['age_band']].apply(pd.to_numeric)

In [ ]:
df_test['family_income'].unique()

In [ ]:
salary_1=['<22,500, >=20,000', '<25,000, >=22,500','<30,000, >=27,500', '<20,000, >=17,500', '<27,500, >=25,000','< 4,000', '<15,000, >=12,500', '<17,500, >=15,000',
       '< 8,000, >= 4,000', '<12,500, >=10,000', '<10,000, >= 8,000']

In [ ]:
df_test['family_income'] = df_test['family_income'].str.replace('>=35,000','35000',regex=False)
df_test['family_income'] = df_test['family_income'].str.replace('<22,500, >=20,000','22500',regex=False)
df_test['family_income'] = df_test['family_income'].str.replace('<25,000, >=22,500','25000',regex=False)
df_test['family_income'] = df_test['family_income'].str.replace('<30,000, >=27,500','30000',regex=False)
df_test['family_income'] = df_test['family_income'].str.replace('<20,000, >=17,500','20000',regex=False)
df_test['family_income'] = df_test['family_income'].str.replace('<27,500, >=25,000','27500',regex=False)
df_test['family_income'] = df_test['family_income'].str.replace('< 4,000','4000',regex=False)
df_test['family_income'] = df_test['family_income'].str.replace('<15,000, >=12,500','15000',regex=False)
df_test['family_income'] = df_test['family_income'].str.replace('<17,500, >=15,000','17500',regex=False)
df_test['family_income'] = df_test['family_income'].str.replace('< 8,000, >= 4,000','8000',regex=False)
df_test['family_income'] = df_test['family_income'].str.replace('<12,500, >=10,000','12500',regex=False)
df_test['family_income'] = df_test['family_income'].str.replace('<10,000, >= 8,000','10000',regex=False)
df_test['family_income'] = df_test['family_income'].str.replace('0    35000\ndtype: object','35000',regex=False)



In [ ]:
df_test[['family_income']] = df_test[['family_income']].apply(pd.to_numeric)


In [ ]:
df_test['status'] = df_test['status'].str.replace('0    Partner\ndtype: object','Partner',regex=False)
df_test['gender'] = df_test['gender'].str.replace('0    Female\ndtype: object','Female',regex=False)

# Merging columns

In [ ]:
df_test['investments']=df_test['Investment_in_Mutual_Fund']+df_test['Investment_Tax_Saving_Bond']+df_test['Investment_in_Commudity']+df_test['Investment_in_Equity']+df_test['Investment_in_Derivative']+df_test['Term_Deposit']
df_test['spending']=df_test['Balance_Transfer']+df_test['Online_Purchase_Amount']
df_test['loans']=df_test['Personal_Loan']+df_test['Home_Loan']
df_test['insurance']=df_test['Life_Insurance']+df_test['Medical_Insurance']

In [ ]:
df_test.drop(columns=['Life_Insurance','Medical_Insurance','Personal_Loan','Investment_in_Mutual_Fund','Investment_Tax_Saving_Bond','Home_Loan','Investment_in_Commudity','Investment_in_Equity','Investment_in_Derivative','Term_Deposit','Balance_Transfer','Online_Purchase_Amount','post_area','region'],inplace=True)
df_test.drop(columns=['post_code','REF_NO'],inplace=True)
df_test.drop(['year_last_moved','children','TVarea'],inplace=True,axis=1)

In [ ]:
cat_cols =list(['status', 'occupation', 'occupation_partner', 'home_status',
       'self_employed', 'self_employed_partner', 'gender',
       ])

In [ ]:
for c in cat_cols:
    print(df_test[c].value_counts())
    print('='*70)

# Label encoding for all the categorical columns

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for x in cat_cols:
    df_test[x] = le.fit_transform(df_test[x].astype(str))
  

# Skewness of columns

In [ ]:
df_test['Portfolio_Balance'].skew(axis=0)

In [ ]:
df_test['Portfolio_Balance']=np.log(x_main['Portfolio_Balance']+75)

In [ ]:
df_test['Portfolio_Balance'].skew(axis=0)

In [ ]:
num_cols=['age_band','family_income','Average_Credit_Card_Transaction','Average_A/C_Balance','investments','spending','loans','insurance']    
skew_values=dict()
for x in num_cols:
    skew_values[x]=df_test[x].skew(axis=0)
        

In [ ]:
pprint(skew_values)

In [ ]:
for x in list(skew_values):
    if skew_values[x]>-0.5 and skew_values[x]<0.5:
        del skew_values[x]

In [ ]:
pprint(skew_values)

In [ ]:
## Log transformation 

for z in skew_values:
    df_test[z]=np.log(df_test[z]+1)


In [ ]:
skew_values_2=dict()
for x in num_cols:
    skew_values_2[x]=df_test[x].skew(axis=0)
for x in list(skew_values_2):
    if skew_values_2[x]>-0.5 and skew_values_2[x]<0.5:
        del skew_values_2[x]
pprint(skew_values_2)

In [ ]:
### box cox transformation 

for z in skew_values_2:
    df_test[z],_=stats.boxcox(df_test[z]+1)

In [ ]:
skew_values_2=dict()
for x in num_cols:
    skew_values_2[x]=df_test[x].skew(axis=0)
for x in list(skew_values_2):
    if skew_values_2[x]>-0.5 and skew_values_2[x]<0.5:
        del skew_values_2[x]
pprint(skew_values_2)

# Train Test Split-Model

In [ ]:

from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler()
df_test[num_cols]=mms.fit_transform(df_test[num_cols])
df_test[num_cols]=mms.fit_transform(df_test[num_cols])

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
submission=pd.DataFrame(logreg2.predict(df_test))


In [ ]:
submission.shape

In [ ]:
to_delete=pd.read_csv(r'C:\Users\Prawin\Documents\ML PROJECT\existing_base_test.csv')

In [ ]:
submission['REF_NO']=to_delete['REF_NO']

In [ ]:
submission=submission.iloc[:,[1,0]]


In [ ]:
submission

In [ ]:
submission.to_csv(r'submission.csv', index=False, header=False)